In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from transformers import DataCollatorWithPadding

In [2]:
import sys
sys.path.append('/jamba')

In [3]:
%load_ext autoreload
%autoreload 2

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
#load Jamba
#tokenizer
from datasets import load_dataset


In [3]:
#write training loop
#train model

In [6]:
from transformers import AutoTokenizer

In [7]:
imdb = load_dataset("imdb")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True,padding=True)
tokenized_imdb = imdb.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# train_dataloader = DataLoader(
#     tokenized_imdb['train'],
#     batch_size=32,
#     shuffle=True,
#     collate_fn=data_collator  

# test_dataloader = DataLoader(
#     tokenized_imdb['test'],
#     batch_size=32,
#     shuffle=False,
#     collate_fn=data_collator
# )

/u/r/i/riyad/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from torch.utils.data import DataLoader

# Step 1: Load the IMDB dataset
imdb = load_dataset("imdb")

# Step 2: Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Step 3: Define preprocessing function
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)  # padding left to collator

# Step 4: Apply preprocessing
tokenized_imdb = imdb.map(preprocess_function, batched=True)

# Step 5: Remove unnecessary columns (especially 'text')
tokenized_imdb = tokenized_imdb.remove_columns(["text"])

# Step 6: Set format for PyTorch
tokenized_imdb.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Step 7: Create dynamic padding collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Step 8: Create dataloaders
train_dataloader = DataLoader(
    tokenized_imdb["train"],
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator
)

test_dataloader = DataLoader(
    tokenized_imdb["test"],
    batch_size=32,
    collate_fn=data_collator
)


/u/r/i/riyad/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
batch = next(iter(train_dataloader))
print({k: v.shape for k, v in batch.items()})

{'input_ids': torch.Size([32, 512]), 'attention_mask': torch.Size([32, 512]), 'labels': torch.Size([32])}


In [9]:
from jamba.model import JambaSequenceClassfication
from jamba.model import  JambaConfig

In [10]:
jambaconfig = JambaConfig(
    dim=512,
    depth=1,
    num_tokens=100,
    d_state=256,
    d_conv=128,
    heads=8,
    num_experts=8,
    num_experts_per_token=2,
    return_embeddings=True,
    num_classes=2,
)
model = JambaSequenceClassfication(jambaconfig)
x = torch.randint(0, 100, (1, 100))
out = model(x)
print(out)

tensor([[[-0.6877, -0.6987],
         [-0.6878, -0.6986],
         [-0.6882, -0.6982],
         [-0.6881, -0.6982],
         [-0.6880, -0.6983],
         [-0.6884, -0.6980],
         [-0.6888, -0.6975],
         [-0.6888, -0.6975],
         [-0.6890, -0.6973],
         [-0.6876, -0.6987],
         [-0.6893, -0.6970],
         [-0.6878, -0.6985],
         [-0.6897, -0.6967],
         [-0.6883, -0.6980],
         [-0.6874, -0.6990],
         [-0.6895, -0.6968],
         [-0.6887, -0.6976],
         [-0.6871, -0.6992],
         [-0.6893, -0.6970],
         [-0.6880, -0.6983],
         [-0.6882, -0.6981],
         [-0.6893, -0.6970],
         [-0.6868, -0.6996],
         [-0.6870, -0.6993],
         [-0.6876, -0.6987],
         [-0.6865, -0.6998],
         [-0.6877, -0.6986],
         [-0.6881, -0.6982],
         [-0.6889, -0.6974],
         [-0.6886, -0.6977],
         [-0.6875, -0.6989],
         [-0.6862, -0.7001],
         [-0.6873, -0.6990],
         [-0.6855, -0.7009],
         [-0.6

In [ ]:
model

Jamba(
  (layers): ModuleList(
    (0): JambaBlock(
      (mamba_layer): MambaBlock(
        (in_proj): Linear(in_features=512, out_features=2048, bias=False)
        (conv1d): Conv1d(1024, 1024, kernel_size=(128,), stride=(1,), padding=(127,), groups=1024)
        (x_proj): Linear(in_features=1024, out_features=544, bias=False)
        (dt_proj): Linear(in_features=32, out_features=1024, bias=True)
        (out_proj): Linear(in_features=1024, out_features=512, bias=False)
      )
      (mamba_moe_layer): MambaMoELayer(
        (mamba): MambaBlock(
          (in_proj): Linear(in_features=512, out_features=2048, bias=False)
          (conv1d): Conv1d(1024, 1024, kernel_size=(128,), stride=(1,), padding=(127,), groups=1024)
          (x_proj): Linear(in_features=1024, out_features=544, bias=False)
          (dt_proj): Linear(in_features=32, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=512, bias=False)
        )
        (moe): MoE(
          (g

In [12]:
output.shape

torch.Size([1, 100, 512])

In [18]:
for batch in train_dataloader:
    print(batch)
    break

{'input_ids': tensor([[  101, 10166,  1010,  ...,     0,     0,     0],
        [  101,  1045,  1005,  ...,     0,     0,     0],
        [  101,  2397,  2028,  ...,     0,     0,     0],
        ...,
        [  101,  2023,  2001,  ...,     0,     0,     0],
        [  101,  3827,  2003,  ...,     0,     0,     0],
        [  101,  2023,  3185,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0,
        0, 1, 1, 1, 1, 0, 1, 0])}


In [14]:
model.to(device)
# Loss function and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 5
for epoch in range(epochs):
    for batch in train_dataloader:
        optimizer.zero_grad()  # Zero the gradients
        inputs = batch['input_ids'].to(device)
        targets = batch['labels'].to(device)
        # Forward pass
        outputs = model(inputs)
        loss = criterion(
            outputs, targets
        )
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

print("Training complete!")


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
from transformers import Trainer
from transformers import TrainingArguments
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")
 
def compute_metrics(eval_pred):
    predictions, label = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions = predictions, references = label)
training_args = TrainingArguments(
    output_dir="./results",
    logging_dir="./logs",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    push_to_hub=False,
    report_to = "none",
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb['train']['input_ids'],
    eval_dataset=tokenized_imdb['test']['input_ids'],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)
trainer.train()